In [1]:
import findspark
findspark.init('/home/gerardo-rodriguez/spark-4.0.0-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('NLP').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/21 11:10:06 WARN Utils: Your hostname, Lanz-Lenovo, resolves to a loopback address: 127.0.1.1; using 192.168.1.145 instead (on interface wlp2s0)
25/08/21 11:10:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/21 11:10:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/21 11:10:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/08/21 11:10:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [6]:
df = spark.createDataFrame([
    (0, 'Hi I heard about Spark'),
    (1, 'I wish java could use case classes'),
    (2, 'Logistic,regression,models,are,neat')
], ['id', 'sentence'])

In [7]:
df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [13]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [14]:
regex_token = RegexTokenizer(inputCol='sentence', outputCol='words', pattern='\\W')

In [15]:
count_tkn = udf(lambda words:len(words), IntegerType())

In [16]:
tokenized = tokenizer.transform(df)
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [18]:
tokenized.withColumn('tokens', count_tkn(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [22]:
regex_tokenized = regex_token.transform(df)
regex_tokenized.withColumn('tokens', count_tkn(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



## StopWordsRemover

In [23]:
from pyspark.ml.feature import StopWordsRemover

In [32]:
sentenceDf = spark.createDataFrame([
    (0,['I','saw','the','green','house']),
    (1,['Mary','had','a','little','lamb'])
], ['id', 'tokens'])

In [33]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')

In [34]:
remover.transform(sentenceDf).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, house]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



## NGram

In [35]:
from pyspark.ml.feature import NGram

In [63]:
word_df = spark.createDataFrame([
    (0, ['Hi', 'I', 'heard', 'about', 'Spark']),
    (1, ['I', 'wish', 'java', 'could', 'use', 'case', 'classes']),
    (2, ['Logistic','regression','models','are','neat'])
], ['id', 'sentence'])

In [65]:
ngram = NGram(n=2, inputCol='sentence', outputCol='grams')

In [69]:
ngram.transform(word_df).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish java, java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



## HasingTF, IDF, Tokenizer

In [72]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF

In [73]:
sentence_df = spark.createDataFrame([
    (0.0, 'Hi I heard about Spark'),
    (0.0, 'I wish java could use case classes'),
    (1.0, 'Logistic,regression,models,are,neat')
], ['label', 'sentence'])

In [74]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
word_data = tokenizer.transform(sentence_df)

In [76]:
word_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic,regression,models,are,neat|[logistic,regression,models,are,neat]     |
+-----+-----------------------------------+------------------------------------------+



In [78]:
hashind_df = HashingTF(inputCol='words', outputCol='rawFeatures')
featurized_data = hashind_df.transform(word_data)

In [81]:
idf = IDF(inputCol='rawFeatures', outputCol='features')
idf_model = idf.fit(featurized_data)

In [87]:
rescaled_data = idf_model.transform(featurized_data)
rescaled_data.select('label', 'features').show(truncate=False)

25/08/21 11:39:04 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
25/08/21 11:39:04 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
25/08/21 11:39:04 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                      |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[18700,19036,33808,66273,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                   |
|0.0  |(262144,[19036,20719,55551,58672,98717,109547,192310],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|1.0 

## countVectorizer

In [88]:
from pyspark.ml.feature import CountVectorizer

In [89]:
df = spark.createDataFrame([
    (0, "a b c".split(' ')),
    (1, "a b b c a".split(' ')),
], ['id', 'words'])

In [90]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [91]:
cv = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF=2.0)
model = cv.fit(df)
result = model.transform(df)
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

